In [7]:
import pandas as pd
import numpy as np

In [8]:
def data_source(pde_base_path, type_list , bu, brand_path, cluster_path, exclusion_list_path):
    
    """
    Extract data from data source.

    Args:
        pde_base_path (str): base path for pde files
        type_list (list): list of pde type e.g. ['haircare', 'skincare', 'vds']
        bu (str): business unit
        brand_path (str): path for brand data
        cluster_path (str): path for cluster data
        exclusion_list_path (str): path for exclusion list data
    
    Return:
        pde_dfs (list): list of pde dataframe(s)
        haircare_list (list): list of attributes selected from haircare
        skincare_list (list): list of attributes selected from skincare
        vds_list (list): list of attributes selected from vds
        brand_data (dataframe): dataframe for brand data
        all_product_details (dataframe): dataframe for product details data

    Example:
        pde_dfs, haircare_list, skincare_list, vds_list, brand_data, all_product_details = data_source( 
            "../PDE_Result_WTCTH_haircare_uda.csv", ['haircare'], 'WTCTH' , '../brand_mapping.csv', "../cluster.csv", "../Exclusion for Need State Name.xlsx")

    """

    exclusion_list = pd.read_excel(exclusion_list_path, engine='openpyxl')
    exclusion_list_df = exclusion_list.groupby('Category')['Attribute Value'].apply(list)
    haircare_exclusion_list = exclusion_list_df['Hair Care']
    skincare_exclusion_list = exclusion_list_df['Skin Care']
    vds_exclusion_list = exclusion_list_df['VDS']
    haircare_list = []
    skincare_list = []
    vds_list = []
    pde_dfs={}

    # Modify attribute lists
    for t in type_list:
        raw_data_path = pde_base_path.format(bu=bu, type=t)
        data = pd.read_csv(raw_data_path)
        attribute_list = list(data.columns)
        attribute_list = [x for x in attribute_list if x not in remove_col_list]

        if t == 'haircare':
            haircare_list = [x for x in attribute_list if x not in haircare_exclusion_list]
        elif t == 'skincare':
            skincare_list = [x for x in attribute_list if x not in skincare_exclusion_list]
        elif t == 'vds':
            vds_list = [x for x in attribute_list if x not in vds_exclusion_list]
        else:
            print('Error in attribute list')

        pde_dfs[t] = data

    # Brand data
    brand_data = pd.read_csv(brand_path)

    # Full details of product 
    all_product_details = pd.read_csv(cluster_path)

    return pde_dfs, haircare_list, skincare_list, vds_list, brand_data, all_product_details



In [9]:
def brand_max(all_product_details, brand_data):

    """
    Get brands with max count for each sub-category.

    Args:
        all_product_details (dataframe): dataframe for product details data
        brand_data (dataframe): dataframe for brand data
    Return:
        brand_max_value (dataframe): dataframe containing max count of brands for each sub-category

    Example:
        brand_max_value = brand_max(all_product_details, brand_data)

    """
    
    brand_full_data = pd.merge(all_product_details, brand_data, on='product_key', how='left')
    brand_max_value = brand_full_data.groupby(column_list+['brand']).size().reset_index(name='counts')
    brand_idx = brand_max_value.groupby(column_list)['counts'].transform(max) == brand_max_value['counts']
    brand_max_value = brand_max_value[brand_idx]
    brand_max_value = brand_max_value.groupby(column_list)['brand'].apply('|'.join).reset_index()
    # brand_max_value.to_csv('/home/brand_max_value.csv')
    return brand_max_value

In [10]:
def haircare_max(all_product_details, haircare_list, haircare_data, brand_max_value):

    """
    Get attributes with top count for haircare product

    Args:
        all_product_details (datafarme): dataframe for product details data
        haircare_list (list): list of attributes selected from haircare
        haircare_data (dataframe):  dataframe of filtered haircare data
        brand_max_value (dataframe): dataframe containing max count of brands for each sub-category

    Return:
        haircare_max_value (dataframe): dataframe containing max count of brands and max count of attributes for each sub-category of type haircare

    Example:
        haircare_max_value = haircare_max(all_product_details, haircare_list, pde_dfs['haircare'], brand_max_value)

    """

    haircare_full_data = pd.DataFrame()
    haircare_max_value = pd.DataFrame()
    haircare_full_data = pd.merge(all_product_details, haircare_data, on='product_key', how='inner')

    for col in haircare_list:
        haircare_full_data[col] = haircare_full_data[col].mask(haircare_full_data[col].ne(1))
        haircare_max_value_temp = haircare_full_data.groupby(column_list)[col].count()
        haircare_max_value = pd.concat([haircare_max_value, haircare_max_value_temp], axis=1)

    top_n = haircare_max_value.apply(lambda s: s.index[s.eq(s.max())].tolist(), axis=1)
    top_n = top_n.apply(lambda s: '|'.join(s))
    haircare_max_value['top n'] = top_n

    haircare_hierarchy = pd.DataFrame(haircare_max_value.index.tolist(), columns=column_list)
    haircare_max_value.reset_index(drop=True, inplace=True)
    haircare_hierarchy.reset_index(drop=True, inplace=True)
    haircare_max_value = pd.concat([haircare_max_value, haircare_hierarchy], axis=1)

    haircare_max_value= pd.merge(haircare_max_value, brand_max_value, on=column_list, how='left')
    haircare_max_value['result'] = haircare_max_value[['brand', 'top n']].apply(lambda x: '|'.join(x.astype(str)), axis=1)
    print(haircare_max_value)
    # hair_max_value.to_csv('/home/hair_max_value.csv')
    return haircare_max_value

In [11]:
# Get attributes with top count for skincare
def skincare_max(all_product_details, skincare_list, skincare_data, brand_max_value):

    """
    Get attributes with top count for skincare product

    Args:
        all_product_details (datafarme): dataframe for product details data
        skincare_list (list): list of attributes selected from skincare
        skincare_data (dataframe):  dataframe of filtered skincare data
        brand_max_value (dataframe): dataframe containing max count of brands for each sub-category

    Return:
        skincare_max_value (dataframe): dataframe containing max count of brands and max count of attributes for each sub-category of type skincare

    Example:
        skincare_max_value = hair_max(all_product_details, skincare_list, pde_dfs['skincare'], brand_max_value)

    """
    
    skincare_max_value = pd.DataFrame()
    skincare_full_data = pd.DataFrame()
    skincare_full_data = pd.merge(all_product_details, skincare_data, on='product_key', how='inner')

    for col in skincare_list:
        skincare_full_data[col] = skincare_full_data[col].mask(skincare_full_data[col].ne(1))
        skincare_max_value_temp = skincare_full_data.groupby(column_list)[col].count()
        skincare_max_value = pd.concat([skincare_max_value, skincare_max_value_temp], axis=1)

    top_n = skincare_max_value.apply(lambda s: s.index[s.eq(s.max())].tolist(), axis=1)
    top_n = top_n.apply(lambda s: '|'.join(s))
    skincare_max_value['top n'] = top_n

    skincare_hierarchy = pd.DataFrame(skincare_max_value.index.tolist(), columns=column_list)
    skincare_max_value.reset_index(drop=True, inplace=True)
    skincare_hierarchy.reset_index(drop=True, inplace=True)
    skincare_max_value = pd.concat([skincare_max_value, skincare_hierarchy], axis=1)

    skincare_max_value= pd.merge(skincare_max_value, brand_max_value, on=column_list, how='left')
    skincare_max_value['result'] = skincare_max_value[['brand', 'top n']].apply(lambda x: '|'.join(x.dropna().astype(str)), axis=1)
    print(skincare_max_value)
    # skincare_max_value.to_csv('/home/skincare_max_value.csv')
    return skincare_max_value

In [12]:
# Get attributes with max 3 count for vds
def vds_max(all_product_details, vds_list, vds_data, brand_max_value):

    """
    Get attributes with top count for vds product

    Args:
        all_product_details (datafarme): dataframe for product details data
        vds_list (list): list of attributes selected from vds
        vds_data (dataframe):  dataframe of filtered vds data
        brand_max_value (dataframe): dataframe containing max count of brands for each sub-category

    Return:
        vds_max_value (dataframe): dataframe containing max count of brands and max count of attributes for each sub-category of type vds

    Example:
        vds_max_value = vds_max(all_product_details, vds_list, pde_dfs['vds'], brand_max_value)

    """
    
    vds_max_value = pd.DataFrame()
    vds_full_data = pd.DataFrame()
    vds_full_data = pd.merge(all_product_details, vds_data, on='product_key', how='inner')

    for col in vds_list:
        vds_full_data[col] = vds_full_data[col].mask(vds_full_data[col].ne(1))
        vds_max_value_temp = vds_full_data.groupby(column_list)[col].count()
        vds_max_value = pd.concat([vds_max_value, vds_max_value_temp], axis=1)

    top_n = vds_max_value.apply(lambda s: s.index[s.eq(s.max())].tolist(), axis=1)
    top_n = top_n.apply(lambda s: '|'.join(s))
    vds_max_value['top n'] = top_n

    vds_hierarchy = pd.DataFrame(vds_max_value.index.tolist(), columns=column_list)
    vds_max_value.reset_index(drop=True, inplace=True)
    vds_hierarchy.reset_index(drop=True, inplace=True)
    vds_max_value = pd.concat([vds_max_value, vds_hierarchy], axis=1)

    vds_max_value= pd.merge(vds_max_value, brand_max_value, on=column_list, how='left')
    vds_max_value['result'] = vds_max_value[['brand', 'top n']].apply(lambda x: '|'.join(x.dropna().astype(str)), axis=1)
    print(vds_max_value)
    # vds_max_value.to_csv('/home/vds_max_value.csv')
    return vds_max_value

In [ ]:
# Main function
# if __name__ == "__main__":

# Set variables
remove_col_list = ['PRODUCT_HIER_1_L1_NAME', 'product_name', 'product_key']
column_list = ['h1_l1_hierarchy_name', 'h1_l2_hierarchy_name', 'h1_l3_hierarchy_name', 'cluster']
brand_path = '../brand_mapping.csv'
cluster_path = "../cluster.csv"
exclusion_list_path = "../Exclusion for Need State Name.xlsx"
pde_base_path = "../PDE_Result_{bu}_{type}_uda.csv"
type_list = ['haircare', 'skincare', 'vds']
bu = 'WTCTH'

# Data analysis
pde_dfs, haircare_list, skincare_list, vds_list, brand_data, all_product_details = data_source(pde_base_path, type_list , bu, brand_path, cluster_path, exclusion_list_path)
brand_max_value = brand_max(all_product_details, brand_data)

if 'haircare' in type_list:
    haircare_max(all_product_details, haircare_list, pde_dfs['haircare'], brand_max_value)

if 'skincare' in type_list:
    skincare_max(all_product_details, skincare_list, pde_dfs['skincare'], brand_max_value)
    
if 'vds' in type_list:
    vds_max(all_product_details, vds_list, pde_dfs['vds'], brand_max_value)